# CS779 Advance DBMS system Term Project | Anjana Nittur

## Setup Spark

In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 53.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840651 sha256=73010fdd6ed8a914c5d140bc630012f679d21f46e919002203361110c7f57a9d
  Stored in directory: /Users/anjananittur/Library/Caches/pip/wheels/2e/d2/18/6f4f20e8332359f7fffceb6828edcc80ef96f86744192a7bb9
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Spark SQL App") \
    .getOrCreate()

24/12/01 13:00:12 WARN Utils: Your hostname, Anjanas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.82 instead (on interface en0)
24/12/01 13:00:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/01 13:00:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/01 13:00:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/01 13:00:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Read and clean datasets 

In [7]:
# Read eBay dataset
ebay_df = spark.read.csv("/Users/anjananittur/Documents/BU/CS779/Term Project/kaggle-dataset/new_ebay_dataset.csv", header=True, inferSchema=True)

# Read Kaggle dataset
kaggle_df = spark.read.csv("/Users/anjananittur/Documents/BU/CS779/Term Project/kaggle-dataset/new_kaggle_dataset.csv", header=True, inferSchema=True)

# Clean datasets: drop nulls and remove duplicates
ebay_cleaned_df = ebay_df.dropna().dropDuplicates()
kaggle_cleaned_df = kaggle_df.dropna().dropDuplicates()

# Show cleaned data
ebay_cleaned_df.show()
kaggle_cleaned_df.show()

+----------+--------------+-------------+-------------+------+--------------+--------------------+-------+--------------------+
|event_type|    product_id|category_code|        brand| price|shipping_price|              seller|ratings|               title|
+----------+--------------+-------------+-------------+------+--------------+--------------------+-------+--------------------+
|      view|item34ba34903e|   smartphone|      samsung|134.98|             1|sellingtech13 (48...|    3.0|NEW LISTINGSamsun...|
|      view|item291780daca|   smartphone|       google|179.99|            12|yywirelesss (145,...|    4.5|Google Pixel 7a G...|
|      view|item5a2dbaae1d|   smartphone|Unknown Brand| 19.99|             8|liquidationkings ...|    1.0|Treo 600 PalmOne ...|
|      view|item1f862566df|   smartphone|        apple|356.99|             9|simplecellllc (32...|    4.5|NEW LISTINGApple ...|
|      view|item2dd485e855|   smartphone|      samsung|  9.99|             9|gadgetaces (598) ...|    2.

24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:15 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------------+----------+----------+-------------------+-------------+-------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|category_code|  brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+-------------+-------+-------+---------+--------------------+
|2019-10-31 20:18:52|  purchase|   1004768|2053013555631882655|   smartphone|samsung| 242.72|566282818|792dae80-feb0-456...|
|2019-10-31 20:19:32|      view|   4804056|2053013554658804075|    headphone|  apple| 160.57|512778900|7f3a7f19-b91f-415...|
|2019-10-31 20:26:01|  purchase|   1004856|2053013555631882655|   smartphone|samsung| 128.42|552335155|68886270-f06a-46d...|
|2019-10-31 20:38:34|      view|   1005230|2053013555631882655|   smartphone|  apple| 875.16|513996417|b86b533e-d767-444...|
|2019-10-31 20:41:29|      view|   1004247|2053013555631882655|   smartphone|  apple| 793.84|554738965|75d39098-e0fb-482...|


## Basic Product and Category Analysis

In [8]:
# Count total records
print("eBay dataset count:", ebay_cleaned_df.count())
print("Kaggle dataset count:", kaggle_cleaned_df.count())

# Summary statistics
ebay_cleaned_df.describe().show()
kaggle_cleaned_df.describe().show()

# Top brands in both datasets
ebay_cleaned_df.groupBy("brand").count().orderBy("count", ascending=False).show(5)
kaggle_cleaned_df.groupBy("brand").count().orderBy("count", ascending=False).show(5)

eBay dataset count: 11701


24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:34 WARN RowBasedKeyValueBatch: Calling spill() on

Kaggle dataset count: 2870619
+-------+----------+--------------+-------------+-------------+------------------+-----------------+--------------------+------------------+--------------------+
|summary|event_type|    product_id|category_code|        brand|             price|   shipping_price|              seller|           ratings|               title|
+-------+----------+--------------+-------------+-------------+------------------+-----------------+--------------------+------------------+--------------------+
|  count|     11701|         11701|        11701|        11701|             11701|            11701|               11701|             11701|               11701|
|   mean|      NULL|          NULL|         NULL|         NULL|102.86843261259166|5.633791983591146|                NULL|3.5555080762328006|                NULL|
| stddev|      NULL|          NULL|         NULL|         NULL|155.24170367451302|5.140186888830991|                NULL|1.3188863958544015|                NULL

24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:38 WARN RowBasedKeyValueBatch: Calling spill() on

+-------+----------+------------------+--------------------+-------------+-------------+------------------+-------------------+--------------------+
|summary|event_type|        product_id|         category_id|category_code|        brand|             price|            user_id|        user_session|
+-------+----------+------------------+--------------------+-------------+-------------+------------------+-------------------+--------------------+
|  count|   2870619|           2870619|             2870619|      2870619|      2870619|           2870619|            2870619|             2870619|
|   mean|      NULL|1375644.9839874257|2.055798651354345...|         NULL|         NULL|443.18452166945883|5.378562257175076E8|                NULL|
| stddev|      NULL|1274656.2773317113|1.801850404132482...|         NULL|         NULL|388.64143118144716|2.099239757216014E7|                NULL|
|    min|      cart|           1000365| 2053013552695869677|    headphone|Unknown Brand|               0.0

24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:05:46 WARN RowBasedKeyValueBatch: Calling spill() on

+-------+------+
|  brand| count|
+-------+------+
|  apple|872585|
|samsung|847941|
| xiaomi|542678|
| huawei|212729|
|   oppo|125607|
+-------+------+
only showing top 5 rows



## Joins

In [10]:
# Join eBay and Kaggle datasets on category_code and brand
joined_df = ebay_cleaned_df.join(
    kaggle_cleaned_df,
    (ebay_cleaned_df.category_code == kaggle_cleaned_df.category_code) &
    (ebay_cleaned_df.brand == kaggle_cleaned_df.brand),
    "inner"
)

# Select specific columns to display
joined_df.select(
    ebay_cleaned_df.category_code,
    ebay_cleaned_df.brand,
    ebay_cleaned_df.price.alias("ebay_price"),
    kaggle_cleaned_df.price.alias("kaggle_price")
).show()


24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:07:47 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------+-------+----------+------------+
|category_code|  brand|ebay_price|kaggle_price|
+-------------+-------+----------+------------+
|   smartphone|samsung|     86.55|      242.72|
|   smartphone|samsung|     74.99|      242.72|
|   smartphone|samsung|    216.65|      242.72|
|   smartphone|samsung|     10.33|      242.72|
|   smartphone|samsung|    114.99|      242.72|
|   smartphone|samsung|     11.27|      242.72|
|   smartphone|samsung|     21.49|      242.72|
|   smartphone|samsung|     172.9|      242.72|
|   smartphone|samsung|     99.99|      242.72|
|   smartphone|samsung|    114.99|      242.72|
|   smartphone|samsung|    149.99|      242.72|
|   smartphone|samsung|    110.99|      242.72|
|   smartphone|samsung|      35.0|      242.72|
|   smartphone|samsung|     144.6|      242.72|
|   smartphone|samsung|     13.81|      242.72|
|   smartphone|samsung|      85.0|      242.72|
|   smartphone|samsung|     84.85|      242.72|
|   smartphone|samsung|    124.55|      

In [11]:
# Create a temporary view for SQL queries
joined_df.createOrReplaceTempView("joined_table")


In [16]:
# Rename columns from both datasets before the join
ebay_cleaned_renamed = ebay_cleaned_df.selectExpr(
    "category_code AS ebay_category_code",
    "brand AS ebay_brand",
    "price AS ebay_price"
)

kaggle_cleaned_renamed = kaggle_cleaned_df.selectExpr(
    "category_code AS kaggle_category_code",
    "brand AS kaggle_brand",
    "price AS kaggle_price"
)

# Perform the join on category_code and brand
joined_df = ebay_cleaned_renamed.join(
    kaggle_cleaned_renamed,
    (ebay_cleaned_renamed.ebay_category_code == kaggle_cleaned_renamed.kaggle_category_code) &
    (ebay_cleaned_renamed.ebay_brand == kaggle_cleaned_renamed.kaggle_brand),
    "inner"
).select(
    "ebay_category_code",
    "ebay_brand",
    "ebay_price",
    "kaggle_price"
)

# Register the result as a temp view for SQL queries
joined_df.createOrReplaceTempView("joined_table")

In [17]:
spark.sql("""
    SELECT 
        ebay_category_code AS category_code,
        ebay_brand AS brand,
        AVG(ebay_price) AS avg_ebay_price,
        AVG(kaggle_price) AS avg_kaggle_price
    FROM joined_table
    WHERE ebay_price > kaggle_price
    GROUP BY ebay_category_code, ebay_brand
    ORDER BY avg_ebay_price DESC
""").show()


24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:12:22 WARN RowBasedKeyValueBatch: Calling spill() on

+-------------+----------+------------------+------------------+
|category_code|     brand|    avg_ebay_price|  avg_kaggle_price|
+-------------+----------+------------------+------------------+
|    headphone|   hifiman|            3384.5|            186.52|
|    headphone|    yamaha| 1626.149230769231| 77.46051282051282|
|       tablet| microsoft|  1098.48142857143| 971.4162585034007|
|   smartphone|      asus|1010.3266666666635|227.30697751873444|
|   smartphone|   oneplus| 929.2111956305719| 625.8165151936498|
|       tablet|     apple| 866.4325000856594| 506.5852852194248|
|   smartphone|     apple| 809.3973219942535| 567.9081603188911|
|   smartphone|    google| 787.5365991867753| 616.6798503330733|
|   smartphone|      sony| 711.7529799107051| 301.3493741414879|
|       tablet|    huawei|             600.0|182.27851465798008|
|   smartphone|     honor| 573.8768187795672|254.47346614373552|
|       tablet|    xiaomi| 506.4951910032148| 346.3848661192431|
|   smartphone|blackberry

## Advanced Analysis

In [21]:
from pyspark.sql.functions import abs, when

# Calculate price differences and suggest action
joined_df = joined_df.withColumn(
    "price_difference", 
    abs(joined_df["ebay_price"] - joined_df["kaggle_price"])  # Use abs from pyspark.sql.functions
).withColumn(
    "suggested_action",
    when(joined_df["ebay_price"] > joined_df["kaggle_price"], "Lower price on eBay")
    .otherwise("Lower price on Kaggle")
)

# Display results for verification
joined_df.select("ebay_category_code", "ebay_brand", "price_difference", "suggested_action").show()


24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 13:15:50 WARN RowBasedKeyValueBatch: Calling spill() on

+------------------+----------+------------------+--------------------+
|ebay_category_code|ebay_brand|  price_difference|    suggested_action|
+------------------+----------+------------------+--------------------+
|        smartphone|   samsung|156.17000000000002|Lower price on Ka...|
|        smartphone|   samsung|167.73000000000002|Lower price on Ka...|
|        smartphone|   samsung|26.069999999999993|Lower price on Ka...|
|        smartphone|   samsung|            232.39|Lower price on Ka...|
|        smartphone|   samsung|            127.73|Lower price on Ka...|
|        smartphone|   samsung|            231.45|Lower price on Ka...|
|        smartphone|   samsung|            221.23|Lower price on Ka...|
|        smartphone|   samsung|             69.82|Lower price on Ka...|
|        smartphone|   samsung|142.73000000000002|Lower price on Ka...|
|        smartphone|   samsung|            127.73|Lower price on Ka...|
|        smartphone|   samsung| 92.72999999999999|Lower price on

In [23]:
from pyspark.sql.functions import sum

# Group by brand and calculate total revenue for both eBay
revenue_df = joined_df.groupBy("ebay_brand").agg(
    sum("ebay_price").alias("total_ebay_revenue"),  # Sum of eBay prices
)

# Display the aggregated results
revenue_df.show()

24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/01 17:56:34 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+--------------------+
|ebay_brand|  total_ebay_revenue|
+----------+--------------------+
|   edifier|             10389.6|
|    marley|             5443.79|
|   philips|  56566.799999999894|
|  logitech|  13544.339999999997|
|    yamaha|  257034.96000000005|
| microsoft|1.4257985369999358E7|
|   bluedio|  2879.2799999999997|
|      meze|  2249.2200000000003|
|     delux|             5475.78|
| panasonic|   249856.5600000021|
|       htc|  2408751.5999999787|
|     jabra|   1097276.399999997|
|     apple|1.467731127149783...|
|       jbl| 1.515909869601699E8|
|    roland|              413.77|
|      ergo|  24029.479999999985|
|       jvc|             8473.99|
|   monster|   8461.750000000002|
|    fostex|              1785.0|
|     intro|   586.2700000000001|
+----------+--------------------+
only showing top 20 rows



## Save file for tableau

In [ ]:
# Save to CSV for Tableau
joined_df.write.csv("output/joined_analysis", header=True, mode="overwrite")

# Save aggregated revenue analysis
joined_df.groupBy("brand").agg(
    sum(joined_df["ebay_price"]).alias("total_ebay_revenue"),
    sum(joined_df["kaggle_price"]).alias("total_kaggle_revenue")
).write.csv("output/revenue_analysis", header=True, mode="overwrite")
